In [4]:
import math
import random
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import seaborn as sns

In [6]:
df = pd.read_csv("ok.csv")
df.columns

/var/folders/y8/cr31wch939v7z0g7_y6bkdzm0000gn/T/ipykernel_85419/41504639.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ok.csv")


Index(['年份', '性别', '年龄', '18岁以下子女数量', '地区', '户口性质', '工作与否', '失业原因', '受教育程度',
       '行业', '个人年收入', '家庭年开支', '生育态度（如果没有政策限制，希望有几个孩子）', '婚姻状况',
       '平等态度（你是否认为男女有同样的晋升空间）', '传统观念（您是否同意男主外女主内）', '性别刻板印象（您是否同意男性能力天生比女性强）',
       '婚姻依赖型（您是否同意干得好不如嫁的好）', '性别偏见（您是否同意经济不景气时应该先解雇女性）',
       '家务态度（您是否同意夫妻应该均等分摊家务）5个维度', '上一周工作时长', '工作自主性'],
      dtype='object')

In [8]:
# 导入数据
df = pd.read_csv("ok.csv")
df = df[df['性别']=='女']  # 只保留女性样本
# 受教育程度
df['edu'] = df['受教育程度'].map({
    '没有受过任何教育': 0,
    '私塾、扫盲班': 1,
    '小学': 2,
    '初中': 3,
    '职业高中': 4,
    '普通高中': 5,
    '中专': 6,
    '技校': 7,
    '大学专科（成人高等教育）': 8,
    '大学专科（正规高等教育）': 9,
    '大学本科（成人高等教育）': 10,
    '大学本科（正规高等教育）': 11,
    '研究生及以上': 12
})
df['dq'] = df['地区'].map({
    '一线城市': {
        '北京': 1,
        '上海': 1,
        '深圳': 1
    },
    '直辖市': {
        '天津': 2,
        '重庆': 2,
        '广东': 2
    },
    '东部沿海地区': {
        '福建': 2,
        '河北': 3,
        '江苏': 2,
        '浙江': 2,
        '山东': 2
    },
    '中西部地区': {
        '安徽': 3,
        '甘肃': 3,
        '广西': 3,
        '贵州': 3,
        '海南': 3,
        '河南': 3,
        '黑龙江': 3,
        '湖北': 3,
        '湖南': 3,
        '吉林': 3,
        '江西': 3,
        '辽宁': 3,
        '内蒙古': 3,
        '宁夏': 3,
        '青海': 3,
        '山西': 3,
        '陕西': 3,
        '四川': 3,
        '西藏': 3,
        '新疆': 3,
        '云南': 3
    }
})
df['hk'] = df['户口性质'].map({
    '城市户口': {
        '城镇常住户口': 1,
        '地级市城区户口': 1,
        '非农户口（城镇户口）': 1,
        '非农业户口': 1,
        '居民户口（例如：户籍制度改革后的广东）': 1,
        '居民户口（以前是非农业户口）': 1,
        '省会城市城区户口': 1,
        '县级市城区户口': 1,
        '直辖市城区户口': 1
    },
    '农村户口': {
        '农业户口': 2,
        '农村户口': 2,
        '非农户口（蓝印户口）': 2,
        '居民户口（以前是农业户口）': 2
    },
    '其他': {
        '海外': 3,
        '集镇或自理口粮户口': 3,
        '军籍': 3,
        '没有户口': 3,
        '其他': 3
    }
})
df['hy'] = df['行业'].map({
    '第一类：党政机关': {
        '党政机关': 1,
        '军队': 1,
        '事业单位': 1,
        '社会团体': 1,
        '社会团体、居/村委会': 1
    },
    '第二类：企业': {
        '企业': 2
    },
    '第三类：无单位/自雇': {
        '无单位/自雇（包括个体户）': 3,
        '无单位/自雇/自办（合伙）企业': 3,
        '其他': 3
    }
})
df['alone'] = df['婚姻状况'].map({
    '与伴侣住在一起': {
        '初婚有配偶': 1,
        '已婚': 1,
        '已婚有配偶': 1,
        '再婚有配偶': 1,
        '同居': 1,
        '丧偶后再婚': 1,
        '离婚后再婚': 1
    },
    '没有与伴侣住在一起': {
        '从未结过婚': 2,
        '分居': 2,
        '分居未离婚': 2,
        '离婚': 2,
        '离婚未再婚': 2,
        '丧偶': 2,
        '丧偶未再婚': 2,
        '未婚': 2
    }
})
df['freetime'] = df['工作自主性'].map({
    '完全不能自主': 1,
    '在很少程度上自主': 2,
    '能在一定程度上自主': 3,
    '完全自主决定': 4
})

df['生育态度（如果没有政策限制，希望有几个孩子）'] = df['syyy'],
df['传统观念（您是否同意男主外女主内）'] = df['ctgn'],
df['性别刻板印象（您是否同意男性能力天生比女性强）'] = df['xbqs'],
df['婚姻依赖型（您是否同意干得好不如嫁的好）'] = df['dlzz'],
df['性别偏见（您是否同意经济不景气时应该先解雇女性）'] = df['xbpj'],
df['家务态度（您是否同意夫妻应该均等分摊家务）'] = df['jwpd']
df['age'] = df['年份'] - df['年龄']  # 年龄
df = df[(df['age'] >= 18) & (df['age'] <= 35)]
df['income'] = df['个人年收入']  # 收入
df['childs'] = ((df['18岁以下子女数量'] != '没有') * (1 - df['A68b'].isna())).astype(int)  # 子女数

df = df[['age', 'edu', 'childs', 'income']]
df = df.fillna(0)

/var/folders/y8/cr31wch939v7z0g7_y6bkdzm0000gn/T/ipykernel_85419/623069832.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("ok.csv")


KeyError: 'syyy'